Some simplified learning based on
https://colab.research.google.com/drive/1WemHvycYcoNTDr33w7p2HL3FF72Nj88i



In [7]:
!pip -q install langchain-groq 
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph 
!pip -q install python-dotenv

In [4]:
!pip show langgraph

Name: langgraph
Version: 0.0.48
Summary: langgraph
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /Users/dave/dev/python-agent-learn/.venv/lib/python3.9/site-packages
Requires: langchain-core
Required-by: 


In [8]:
import os
from dotenv import load_dotenv
load_dotenv("../.env", override=True)

True

In [9]:
from pprint import pprint

In [75]:
import asyncio
from typing import Any, Dict, List
from uuid import UUID

from langchain.callbacks.base import AsyncCallbackHandler, BaseCallbackHandler
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.outputs import LLMResult


class TokenPrintingCallback(AsyncCallbackHandler):
    """Async callback handler that can be used to handle callbacks from langchain.
      Callbacks are documented here: https://python.langchain.com/v0.1/docs/modules/callbacks/async_callbacks/
    """

    async def on_chat_model_start(self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs: Any) -> Any:
        pass
        # return await super().on_chat_model_start(serialized, messages, run_id=run_id, parent_run_id=parent_run_id, tags=tags, metadata=metadata, **kwargs)

    async def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> None:
        pass

    async def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        total_tokens = response.llm_output.get("token_usage").get("total_tokens")
        print(f"\033[95mUsed total tokens: {total_tokens}\033[0m")


In [76]:

from langchain_groq import ChatGroq

# from typing import Any, Optional, Union  # Import Any from typing module
# from uuid import UUID
# from langchain_core.outputs import GenerationChunk, ChatGenerationChunk

GROQ_LLM = ChatGroq(
            model="llama3-70b-8192",
            callbacks=[TokenPrintingCallback()]
        )

GROQ_LLM.invoke("hello").content


Used total tokens: 36


"Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [77]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [78]:
def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)

In [79]:
#Categorize EMAIL
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Email Categorizer Agent You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Conduct a comprehensive analysis of the email provided and categorize into one of the following categories:
        price_equiry - used when someone is asking for information about pricing \
        customer_complaint - used when someone is complaining about something \
        product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
        customer_feedback - used when someone is giving feedback about a product \
        off_topic when it doesnt relate to any other category \


            Output a single cetgory only from the types ('price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic') \
            eg:
            'price_enquiry' \

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email"],
)

email_category_generator = prompt | GROQ_LLM  | StrOutputParser()


EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

result = email_category_generator.invoke({"initial_email": EMAIL})

print(result)

Used total tokens: 251
'customer_feedback'


In [80]:
## Write Draft Email
draft_writer_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Writer Agent take the INITIAL_EMAIL below  from a human that has emailed our company email address, the email_category \
            that the categorizer agent gave it and the research from the research agent and \
            write a helpful email in a thoughtful and friendly way.

            If the customer email is 'off_topic' then ask them questions to get more information.
            If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
            If the customer email is 'price_equiry' then try to give the pricing info they requested.

            You never make up information that hasn't been provided by the research_info or in the initial_email.
            Always sign off the emails in appropriate manner and from Sarah the Resident Manager.

            Return the email a JSON with a single key 'email_draft' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    RESEARCH_INFO: {research_info} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category","research_info"],
)

draft_writer_chain = draft_writer_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None

print(draft_writer_chain.invoke({"initial_email": EMAIL, "email_category":email_category,"research_info":research_info}))

Used total tokens: 425
{'email_draft': "Dear Paul,\n\nThank you so much for taking the time to share your wonderful feedback about your recent stay with us! We're thrilled to hear that our staff were able to make a positive impact on your experience.\n\nWe truly appreciate your kind words and are grateful for customers like you who make our job so rewarding.\n\nThank you again for choosing to stay with us, and we hope to have the pleasure of welcoming you back soon.\n\nBest regards,\nSarah, Resident Manager"}


In [81]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [82]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        initial_email: email
        email_category: email category
        draft_email: LLM generation
        final_email: LLM generation
        research_info: list of documents
        info_needed: whether to add search info
        num_steps: number of steps
    """
    initial_email : str
    email_category : str
    draft_email : str
    final_email : str
    research_info : List[str]
    info_needed : bool
    num_steps : int
    draft_email_feedback : dict

In [83]:
def categorize_email(state):
    """take the initial email and categorize it"""
    print("---CATEGORIZING INITIAL EMAIL---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1

    email_category = email_category_generator.invoke({"initial_email": initial_email})
    print(email_category)
    # save to local disk
    write_markdown_file(email_category, "email_category")

    return {"email_category": email_category, "num_steps":num_steps}

In [85]:
def draft_email_writer(state):
    print("---DRAFT EMAIL WRITER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email = draft_writer_chain.invoke({"initial_email": initial_email,
                                     "email_category": email_category,
                                     "research_info":research_info})
    print(draft_email)
    # print(type(draft_email))

    email_draft = draft_email['email_draft']
    write_markdown_file(email_draft, "draft_email")

    return {"draft_email": email_draft, "num_steps":num_steps}

In [86]:
def state_printer(state):
    """print the state"""
    print("---STATE PRINTER---")
    print(f"Initial Email: {state['initial_email']} \n" )
    print(f"Email Category: {state['email_category']} \n")
    print(f"Draft Email: {state['draft_email']} \n" )
    print(f"Final Email: {state['final_email']} \n" )
    print(f"Research Info: {state['research_info']} \n")
    print(f"Info Needed: {state['info_needed']} \n")
    print(f"Num Steps: {state['num_steps']} \n")
    return

In [87]:
def route_to_research(state):
    """
    Route email to web search or not.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """

    print("---ROUTE TO RESEARCH---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]



    return "draft_email"

In [88]:

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("categorize_email", categorize_email) # categorize email
# workflow.add_node("research_info_search", research_info_search) # web search
workflow.add_node("state_printer", state_printer)
workflow.add_node("draft_email_writer", draft_email_writer)
# workflow.add_node("analyze_draft_email", analyze_draft_email)
# workflow.add_node("rewrite_email", rewrite_email)
# workflow.add_node("no_rewrite", no_rewrite)



In [89]:
workflow.set_entry_point("categorize_email")

workflow.add_conditional_edges(
    "categorize_email",
    route_to_research,
    {
        # "research_info": "research_info_search",
        "draft_email": "draft_email_writer",
    },
)


workflow.add_edge("draft_email_writer","state_printer")

workflow.add_edge("state_printer", END)

In [93]:
app = workflow.compile()

In [95]:
EMAIL1 = """HI there, \n
I am emailing to find out the current price of Bitcoin. \n

Can you please help me/

Thanks,
John
"""

EMAIL2 = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

EMAIL3 = """HI there, \n
I am emailing to say that the resort weather was way to cloudy and overcast. \n
I wanted to write a song called 'Here comes the sun but it never came'

What should be the weather in Arizona in April?

I really hope you fix this next time.

Thanks,
George
"""

EMAIL4 = """HI there, \n
Why can't I get to sing?

Thanks,
Ringo
"""

EMAIL5 = """HI there, \n
Thanks for confirming my booking

Thanks,
Ringo
"""

In [96]:
# run the agent
inputs = {"initial_email": EMAIL1,"research_info": None, "num_steps":0}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")

---CATEGORIZING INITIAL EMAIL---
Used total tokens: 242
'price_enquiry'
---ROUTE TO RESEARCH---
'Finished running: categorize_email:'
---DRAFT EMAIL WRITER---
Used total tokens: 400
{'email_draft': "Hi John, \n\nThank you for reaching out to us about the current price of Bitcoin. Unfortunately, as the price of Bitcoin is constantly changing, I don't have the most up-to-date information to provide you. I recommend checking a reliable cryptocurrency website or exchange for the most current pricing information.\n\nIf you have any other questions or need further assistance, please don't hesitate to ask.\n\nBest regards,\nSarah, Resident Manager"}
'Finished running: draft_email_writer:'
---STATE PRINTER---
Initial Email: HI there, 

I am emailing to find out the current price of Bitcoin. 


Can you please help me/

Thanks,
John
 

Email Category: 'price_enquiry' 

Draft Email: Hi John, 

Thank you for reaching out to us about the current price of Bitcoin. Unfortunately, as the price of Bitc